# 株価データの取得と保存
このノートブックでは、companiesテーブルに登録された企業のYahoo Financeティッカーを用いて株価データを取得し、PostgreSQLのstock_pricesテーブルに保存します。

In [ ]:
## ライブラリのインポート

In [1]:
import os
from datetime import datetime
import pandas as pd
import yfinance as yf
from sqlalchemy import create_engine, text
from sqlalchemy.engine import URL

## データベースからティッカー一覧を取得

In [2]:
# .envから接続情報を読み込み
from dotenv import load_dotenv
load_dotenv('../.env')

# SQLAlchemy用の接続URLを作成
database_url = URL.create(
    drivername='postgresql+psycopg2',
    username=os.getenv('POSTGRES_USER'),
    password=os.getenv('POSTGRES_PASSWORD'),
    host=os.getenv('POSTGRES_HOST', 'localhost'),
    port=os.getenv('POSTGRES_PORT', '5432'),
    database=os.getenv('POSTGRES_DB'),
)
engine = create_engine(database_url)

# companiesテーブルから company_id と security_code を取得
sql = """
    SELECT company_id, security_code
    FROM companies
    WHERE security_code IS NOT NULL
"""
df_companies = pd.read_sql(sql, engine)

# Yahoo Finance用ティッカーを生成（先頭4桁 + '.T'）
df_companies['yahoo_ticker'] = (
    df_companies['security_code']
      .astype(str)
      .str.zfill(4)       # 万一3桁以下なら4桁にゼロ埋め
      .str.slice(0, 4)    # 先頭4文字のみ
      + '.T'
)

tickers = df_companies['yahoo_ticker'].tolist()
print(f'{len(tickers)} tickers loaded (e.g. {tickers[:5]})')

1264 tickers loaded (e.g. ['130A.T', '135A.T', '137A.T', '138A.T', '1401.T'])


## yfinanceで株価データを取得

In [ ]:
import time
import pandas as pd
import yfinance as yf
from datetime import datetime

# --- 設定 ---
start = "2015-01-01"
end   = datetime.today().strftime("%Y-%m-%d")
max_retries   = 3
sleep_seconds = 1

all_data       = []
failed_tickers = []

def fetch_ticker_data(ticker: str) -> pd.DataFrame | None:
    """
    単一ティッカーのデータを取得。
    RateLimit (Too Many Requests) はリトライ、
    その他のエラー・空データは即失敗扱い。
    """
    for attempt in range(1, max_retries + 1):
        try:
            df = yf.download(
                ticker,
                start=start,
                end=end,
                threads=False,       # 単一ならスレッド不要
                auto_adjust=False,   # Adj Close列も取得
                progress=False
            )
            # データが何もない場合は失敗
            if df.empty:
                return None

            df = df.reset_index()
            df["ticker"] = ticker
            return df

        except Exception as e:
            msg = str(e)
            # RateLimitエラー判定
            if "Rate limited" in msg or "Too Many Requests" in msg:
                print(f"[RateLimit] {ticker} retry {attempt}/{max_retries}")
                time.sleep(sleep_seconds)
                continue

            # その他のエラーは即時失敗
            print(f"[Error] {ticker} -> {e}")
            break

    # リトライ切れ、または空データ
    return None

# --- 全ティッカーをループ ---
for ticker in tickers:
    df_t = fetch_ticker_data(ticker)
    if df_t is None:
        failed_tickers.append(ticker)
    else:
        all_data.append(df_t)

# --- 結果サマリ ---
print(f"✅ Success: {len(all_data)} / {len(tickers)} tickers")
print(f"❌ Failed ({len(failed_tickers)}): {failed_tickers}")

if not all_data:
    raise RuntimeError("No data downloaded: cannot build stock_df")

# --- DataFrame 結合 ---
stock_df = pd.concat(all_data, ignore_index=True)
print("✅ stock_df created:", stock_df.shape)

In [3]:
# 単一ティッカー取得テスト
import yfinance as yf
from datetime import datetime

# テストしたい銘柄コード（例：トヨタ自動車）
test_ticker = "2427.T"

# 取得期間（上場日以降 or 任意の開始日～今日まで）
start_date = "2015-01-01"
end_date   = datetime.today().strftime("%Y-%m-%d")

# データ取得
df = yf.download(
    test_ticker,
    start=start_date,
    end=end_date,
    threads=False,       # 単一ならスレッド不要
    auto_adjust=False,   # 調整前OHLCとAdj Closeが取れる
    progress=False
)

# 結果確認
print(f"Fetched {len(df)} rows for {test_ticker}")
print(df.head())
print(df.tail())


1 Failed download:
['2427.T']: YFTzMissingError('possibly delisted; no timezone found')


Fetched 0 rows for 2427.T
Empty DataFrame
Columns: [(Adj Close, 2427.T), (Close, 2427.T), (High, 2427.T), (Low, 2427.T), (Open, 2427.T), (Volume, 2427.T)]
Index: []
Empty DataFrame
Columns: [(Adj Close, 2427.T), (Close, 2427.T), (High, 2427.T), (Low, 2427.T), (Open, 2427.T), (Volume, 2427.T)]
Index: []
